In [1]:
!pip install --upgrade pip

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
# !pip install torchcodec
!pip install -U typing_extensions

import torch; torch._dynamo.config.recompile_limit = 64;

In [3]:
%%capture
!pip install --no-deps --upgrade timm # Only for Gemma 3N

In [4]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

model, processor = FastVisionModel.from_pretrained(
    "blind-assist/gemma-3n-4b-finetune-8500",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Gemma3N patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.432 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [5]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 239.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 224.4 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3━━━━━ 0/2 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]


In [ ]:
from huggingface_hub import login
import os

# Paste your token here (get it from: https://huggingface.co/settings/tokens)
HF_TOKEN = "" 

login(token=HF_TOKEN)

In [7]:
!pip uninstall torchcodec -y

In [8]:
import cv2
import numpy as np
from PIL import Image

def downsample_video(video_path, num_frames=2):
    """Extracts evenly spaced frames for VLM context."""
    
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0: return []
    
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []
    for i in indices:
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, image = vidcap.read()
        if success:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(image))
    vidcap.release()
    return frames

In [9]:
from unsloth import get_chat_template

processor = get_chat_template(
    processor,
    "gemma-3n"
)

In [10]:
import os
import time
import csv
from pathlib import Path
from transformers import TextStreamer

# Enable Unsloth optimized inference
FastVisionModel.for_inference(model)

Gemma3nForConditionalGeneration(
  (model): Gemma3nModel(
    (vision_tower): TimmWrapperModel(
      (timm_model): MobileNetV5Encoder(
        (conv_stem): ConvNormAct(
          (conv): Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
          (bn): RmsNormAct2d(
            (drop): Identity()
            (act): GELU(approximate='tanh')
          )
        )
        (blocks): Sequential(
          (0): Sequential(
            (0): EdgeResidual(
              (conv_exp): Conv2dSame(64, 256, kernel_size=(3, 3), stride=(2, 2), bias=False)
              (bn1): RmsNormAct2d(
                (drop): Identity()
                (act): GELU(approximate='tanh')
              )
              (aa): Identity()
              (se): Identity()
              (conv_pwl): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn2): RmsNormAct2d(
                (drop): Identity()
                (act): Identity()
              )
              (drop_path): Identity()
       

In [11]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A40. Max memory = 44.432 GB.
14.9 GB of memory reserved.


In [12]:
# Install/upgrade Gradio
!pip install gradio --upgrade -q

In [ ]:
import gradio as gr
from transformers import TextIteratorStreamer
from threading import Thread

def process_video_streaming(video_path, custom_instruction=""):
    """Process video and stream the caption in real-time."""
    
    # Use default instruction if none provided
    if not custom_instruction.strip():
        instruction = (
            "Given the visual input from the user's forward perspective, "
            "generate exactly one short sentence to guide a visually impaired user "
            "by identifying critical obstacles or landmarks, describing their locations "
            "using clock directions relative to the user (12 o'clock is straight ahead), "
            "including relevant details such as size, material, or distance, and giving "
            "one clear action, while prioritizing immediate safety and avoiding any extra explanation."
        )
    else:
        instruction = custom_instruction
    
    if video_path is None:
        yield "⚠️ Please upload a video first."
        return
    
    try:
        # Extract frames
        frames = downsample_video(video_path)
        
        # Prepare message
        messages = [{"role": "user", "content": [{"type": "text", "text": instruction}]}]
        for img in frames:
            messages[0]["content"].append({"type": "image", "image": img})
        
        # Process inputs
        input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = processor(
            images=frames,
            text=input_text,
            add_special_tokens=False,
            return_tensors="pt",
        ).to("cuda")
        
        # Setup streamer for real-time output
        streamer = TextIteratorStreamer(
            processor.tokenizer,
            skip_special_tokens=True,
            skip_prompt=True
        )
        
        # Generation config
        gen_kwargs = {
            **inputs,
            "max_new_tokens": 128,
            "use_cache": True,
            "temperature": 1.0,
            "top_p": 0.95,
            "top_k": 64,
            "streamer": streamer,
        }
        
        # Generate in thread
        thread = Thread(target=model.generate, kwargs=gen_kwargs)
        thread.start()
        
        # Stream output
        generated = ""
        for text in streamer:
            generated += text
            clean = generated.strip()
            
            # Extract model response
            if "model" in clean:
                clean = clean.split("model")[-1].strip()
            elif "assistant" in clean.lower():
                clean = clean.split("assistant")[-1].strip()
            
            clean = " ".join(clean.split())
            yield clean
        
        thread.join()
        
    except Exception as e:
        yield f"❌ Error: {str(e)}"

# Create the interface
with gr.Blocks(title="Video Guidance Demo") as demo:
    gr.Markdown("# 🎥 Video Guidance Generator")
    gr.Markdown("Upload a video to generate navigation guidance for visually impaired users")
    
    with gr.Row():
        with gr.Column():
            video = gr.Video(label="Upload Video", height=400)
            instruction = gr.Textbox(
                label="Custom Instruction (optional)",
                placeholder="Leave empty for default...",
                lines=3
            )
            btn = gr.Button("🚀 Generate", variant="primary")
        
        with gr.Column():
            output = gr.Textbox(
                label="Generated Guidance",
                lines=15,
                placeholder="Guidance will appear here..."
            )
    
    btn.click(process_video_streaming, inputs=[video, instruction], outputs=output)

# Launch
demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f926a4dfb637da9dd5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[matroska,webm @ 0x7fbf4c9612c0] File ended prematurely at pos. 170656 (0x29aa0)
[opus @ 0x7fbf4c09b9c0] Error parsing Opus packet header.
